#get webpage 'finviz.com' website screener as screen shot headless

- open website inside firefox via thirtyfour / tokio

- load crates

In [6]:
:dep eyre
:dep color-eyre = "0.6.2"
:dep serde_json
:dep thirtyfour = "0.31.0"
:dep tokio = "1.28.2"

In [ ]:
//     cargo run --example tokio_async

use thirtyfour::prelude::*;
use thirtyfour::{DesiredCapabilities, WebDriver};

use tokio::time::*;

//pub use common::capabilities::firefox::FirefoxCapabilities;

fn call_website() -> color_eyre::Result<()> {
    let rt: tokio::runtime::Runtime = tokio::runtime::Builder::new_current_thread().enable_all().build()?;
    rt.block_on(run())
}

async fn run() -> color_eyre::Result<()> {
    // The use of color_eyre gives much nicer error reports, including making
    // it much easier to locate where the error occurred.
    color_eyre::install()?;

//https://docs.rs/thirtyfour/0.31.0/thirtyfour/common/capabilities/firefox/struct.FirefoxCapabilities.html
    let mut caps: thirtyfour::FirefoxCapabilities = DesiredCapabilities::firefox();
    caps.add_firefox_arg("MOZ_REMOTE_SETTINGS_DEVTOOLS=1");
    caps.set_headless();

    let driver: WebDriver = WebDriver::new("http://localhost:4444", caps).await?;
    
    // Navigate to target webpage
    driver.goto("https://www.finviz.com").await?;
    
    //wait for loading website 3 sec
    tokio::time::sleep(Duration::from_secs(3)).await;
    
    // Always explicitly close the browser. There are no async destructors.
    driver.quit().await?;

    //driver.close_window().await?;
    //driver.close_window().await?;
    Ok(())
}

- get website title

In [22]:
//     cargo run --example tokio_async

use thirtyfour::prelude::*;
use thirtyfour::{DesiredCapabilities, WebDriver};
use thirtyfour::common::capabilities::firefox::*;

use tokio::time::*;

//pub use common::capabilities::firefox::FirefoxCapabilities;

fn call_website() -> color_eyre::Result<()> {
    let rt: tokio::runtime::Runtime = tokio::runtime::Builder::new_current_thread().enable_all().build()?;
    rt.block_on(run())
}

async fn run() -> color_eyre::Result<()> {

    let target_webpage= String::from("https://www.finviz.com");
    let website_title = String::from("FINVIZ.com - Stock Screener");
    let gecko_url_port = String::from("http://localhost:4444");

    // The use of color_eyre gives much nicer error reports, including making
    // it much easier to locate where the error occurred.
    color_eyre::install()?;

    //https://docs.rs/thirtyfour/0.31.0/thirtyfour/common/capabilities/firefox/struct.FirefoxCapabilities.html
    let mut caps: thirtyfour::FirefoxCapabilities = DesiredCapabilities::firefox();
    caps.add_firefox_arg("MOZ_REMOTE_SETTINGS_DEVTOOLS=1");
    
    caps.set_headless();
    let log_level:LogLevel = LogLevel::Trace;
    caps.set_log_level(log_level);

    let driver: WebDriver = WebDriver::new(&gecko_url_port, caps).await?;
    
    // Navigate to target webpage
    driver.goto(target_webpage).await?;
    
    //wait for loading website 3 sec
    tokio::time::sleep(Duration::from_secs(3)).await;

    driver.current_url();
    assert_eq!(driver.title().await?, website_title);
    
    // close windows
    // `thirtyfour::session::handle::SessionHandle::close`: 
    // This method has been renamed to close_window()
    driver.close_window().await?;


    // Always explicitly close the browser. There are no async destructors.
    driver.quit().await?;

    

    //driver.close_window().await?;
    //driver.close_window().await?;
    Ok(())
}

In [10]:
call_website();

NoScript - prevents the consent banner from loading

# [example from github - firefox_preferences.rs](https://github.com/stevepryde/thirtyfour/blob/main/thirtyfour/examples/firefox_preferences.rs)

In [7]:
//! Requires geckodriver running on port 4444:
//!
//!     geckodriver --port=4444
//!
//! Run as follows:
//!
//!     cargo run --example firefox_preferences

use thirtyfour::common::capabilities::firefox::FirefoxPreferences;
use thirtyfour::{FirefoxCapabilities, WebDriver};

#[tokio::main]
async fn test_ff_pref() -> color_eyre::Result<()> {
    // The use of color_eyre gives much nicer error reports, including making
    // it much easier to locate where the error occurred.
    color_eyre::install()?;

    let user_agent = "Custom";

    // Set user agent via Firefox preferences.
    let mut prefs = FirefoxPreferences::new();
    prefs.set_user_agent(user_agent.to_string())?;
    //prefs.FirefoxCapabilities

    let mut caps = FirefoxCapabilities::new();
    caps.set_preferences(prefs)?;

    let driver = WebDriver::new("http://localhost:4444", caps).await?;
    driver.goto("https://www.google.com").await?;

    // Get the user agent and verify.
    let js_user_agent: String =
        driver.execute(r#"return navigator.userAgent;"#, Vec::new()).await?.convert()?;
    assert_eq!(&js_user_agent, user_agent);

    // Always explicitly close the browser. There are no async destructors.
    driver.close_window().await?;
    Ok(())
}


In [8]:
test_ff_pref();